- vae
    - 连续的隐空间：$z$
    - encoder 的输出，一个概率分布的参数（通常是高斯分布的均值 μ 和方差 σ）
- vqvae: Vector Quantization
    - 离散的码本 Codebook（隐空间）
        - 码本 (Codebook) / 嵌入空间: $\mathcal{E} = {e_i}_{i=1}^K$, 其中 $e_i \in \mathbb{R}^D$ 是码本中的第 $i$ 个向量（码字），共 $K$ 个码字，每个维度为 $D$。
    - encoder 的输出，输出一个或多个特征向量 $z_e(x)$ 

### vqvae

- 编码 (Encoding) 编码器 $E$ 将输入 $x$ 映射到一个连续的特征图（或向量）$z_e(x)$。
    - $z_e(x) \in \mathbb{R}^{h \times w \times D}$ (以图像为例)。
$$
z_e(x) = E(x) 
$$
- 向量量化 (Vector Quantization) 对于 $z_e(x)$ 中的每一个向量（例如，在特征图的每个空间位置上的 $D$ 维向量），我们找到码本 $\mathcal{E}$ 中欧几里得距离最近的那个码字 $e_k$。
    - 这个过程将连续的 $z_e(x)$ 映射到一个离散的索引网格。然后，我们用找到的码字 $e_k$ 来替换原来的 $z_e(x)$，得到量化后的特征图 $z_q(x)$。
$$
k = \arg\min_{j \in \{1, ..., K\}} \| z_e(x) - e_j \|_2^2 
$$
- 解码 (Decoding) 解码器 $D$ 接收量化后的 $z_q(x)$ 并重建输入数据 $\hat{x}$。
$$
\hat{x} = D(z_q(x)) 
$$

$$
 \mathcal{L} = \underbrace{\|x - \hat{x}\|_2^2}_{\text{重建损失}} + \underbrace{\| \text{sg}[z_e(x)] - e_k \|_2^2}_{\text{码本损失}} + \underbrace{\beta \| z_e(x) - \text{sg}[e_k] \|_2^2}_{\text{承诺损失}} 
$$

- Reconstruction Loss: $ \mathcal{L}_{\text{rec}} = \|x - D(z_q(x))\|_2^2 $
- Codebook Loss: $ \mathcal{L}_{\text{codebook}} = \| \text{sg}[z_e(x)] - e_k \|_2^2 $
    - 这个损失的目标是更新码本中的向量 $e_i$
    - sg 代表 stop-gradient (停止梯度) 操作，即 sg[v] = v，但在反向传播时其梯度为零。这意味着此项的梯度只会流向码本 $e$，而不会影响编码器 $E$。它把码字 $e$ “拉向”编码器的输出。
- Commitment Loss: $ \mathcal{L}_{\text{commit}} = \beta \| z_e(x) - \text{sg}[e] \|_2^2 $
    - 这里的 sg 作用于码本 $e$。这意味着此项的梯度只会流向编码器 $E$。它把编码器的输出 $z_e(x)$ “拉向”它所选择的码字 $e$。$\beta$ 是一个超参数，用于控制此项的权重。
- 梯度传递的技巧
    - 在反向传播计算重建损失的梯度时，由于 $z_q(x)$ 的存在，梯度无法从解码器传到编码器。VQ-VAE 使用直通估计器 (Straight-Through Estimator, STE) 来解决这个问题。在计算 $\frac{\partial \mathcal{L}_{\text{rec}}}{\partial z_e(x)}$ 时，我们直接将 $z_q(x)$ 的梯度复制给 $z_e(x)$：
    - 这相当于在反向传播时，假设量化操作是一个恒等函数。

$$
 \frac{\partial \mathcal{L}_{\text{rec}}}{\partial z_e(x)} \approx \frac{\partial \mathcal{L}_{\text{rec}}}{\partial z_q(x)} 
$$